In [21]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.nn import Module, Sequential
from torch.utils.data import DataLoader
from torch import optim
from tqdm import tqdm
import pandas as pd

### Descărcare și normalizare date

In [22]:
image_to_tensor = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    transforms.Lambda(torch.flatten)
])
train_set = torchvision.datasets.MNIST(
    root='./training',
    train=True,
    download=True,
    transform=image_to_tensor,
)

test_set = torchvision.datasets.MNIST(
    root='./test',
    train=False,
    download=True,
    transform=image_to_tensor
)

### Definirea modelului

In [23]:
class MyNN(Module):
    def __init__(self, input_size, num_classes):
        super(MyNN, self).__init__()
        self.model = Sequential(
            torch.nn.Linear(input_size, 256),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(256, 100),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(100, num_classes)
        )
    def forward(self, x):
        return self.model(x)

### Dataloaders, optimizatori, hiperparametrii

In [24]:
# HYPERPARAMETERS
INPUT_SIZE = 784
OUTPUT_CLASSES = 10
LEARNING_RATE = 0.001
EPOCHS = 125
BATCH_SIZE = 32

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True,drop_last=True,num_workers=4)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True, drop_last=True,num_workers=4)

device = torch.device('cuda')
model = MyNN(INPUT_SIZE, OUTPUT_CLASSES).to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)


In [25]:
def check_accuracy(loader, model: Module):
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for image, target in loader:
            image, target = image.to(device), target.to(device)
            output = model(image)
            pred = output.argmax(dim=1, keepdim=True)
            num_correct += pred.eq(target.view_as(pred)).sum().item()
            num_samples += image.size()[0]
            
    model.train()
    return num_correct / num_samples

In [26]:
def generate_submission(test_set, model: Module):
    model.eval()
    
    with torch.no_grad():
        for i, (image, _) in enumerate(test_set):
            image = image.to(device)
            output = model(image)
            pred = output.argmax(dim=0)
            submission_data["ID"].append(i)
            submission_data["target"].append(pred.item())
    data_frame = pd.DataFrame(submission_data)
    data_frame.to_csv('submission.csv', index=False)
    model.train()

### Antrenarea și uploadarea modelului

In [27]:
submission_data = {
    "ID": [],
    "target": [],
}

for epoch in enumerate(tqdm(range(EPOCHS))):
    model.train()
    for _, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        scores = model(images)
        loss = criterion(scores, labels)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
print(f'Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}%')
print(f'Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}%')
generate_submission(test_set, model)

100%|██████████| 125/125 [24:33<00:00, 11.79s/it]


Accuracy on training set: 99.78%
Accuracy on test set: 98.07%
